# Regresión con Redes Neuronales: California Housing

En este notebook exploraremos cómo entrenar diferentes arquitecturas de redes neuronales para un problema de regresión utilizando el dataset **California Housing**. 

Utilizaremos:
- **TensorFlow** y **Keras** para construir los modelos.
- **scikit-learn** para el preprocesamiento de datos.
- **TensorBoard** para visualizar y comparar el rendimiento de las diferentes arquitecturas.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import datetime
import os

print(f"TensorFlow version: {tf.__version__}")

## 1. Carga y Preparación de Datos

In [ ]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

print(f"X_train shape: {X_train.shape}")

## 2. Configuración de TensorBoard

Definiremos una función para crear los callbacks de TensorBoard para cada modelo.

In [ ]:
root_logdir = os.path.join(os.getcwd(), "logs")

def get_run_logdir(model_name):
    run_id = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    return os.path.join(root_logdir, f"{model_name}_{run_id}")

## 3. Definición y Entrenamiento de Arquitecturas

Probaremos tres modelos:
1. **Baseline**: Una única neurona (equivalente a regresión lineal).
2. **Simple**: Una capa oculta con 30 neuronas.
3. **Profunda**: Dos capas ocultas (30 y 20 neuronas).

In [ ]:
def build_and_train_model(name, layers_config, input_shape=X_train.shape[1:]):
    model = keras.models.Sequential(name=name)
    model.add(keras.layers.Input(shape=input_shape))
    
    for units in layers_config:
        model.add(keras.layers.Dense(units, activation="relu"))
    
    model.add(keras.layers.Dense(1)) # Capa de salida para regresión
    
    model.compile(loss="mse", optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=["mae"])
    
    logdir = get_run_logdir(name)
    tensorboard_cb = keras.callbacks.TensorBoard(logdir)
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
    
    history = model.fit(X_train, y_train, epochs=100, 
                        validation_data=(X_valid, y_valid),
                        callbacks=[tensorboard_cb, early_stopping_cb], 
                        verbose=0)
    return model, history

In [ ]:
print("Entrenando Modelo Baseline...")
model_baseline, hist_baseline = build_and_train_model("Model_Baseline", [])

print("Entrenando Modelo Simple (1 capa oculta)...")
model_simple, hist_simple = build_and_train_model("Model_Simple", [30])

print("Entrenando Modelo Deep (2 capas ocultas)...")
model_deep, hist_deep = build_and_train_model("Model_Deep", [30, 20])

print("¡Entrenamiento completado!")

## 4. Visualización en TensorBoard

Descomenta y ejecuta la siguiente celda para abrir TensorBoard dentro del notebook.

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir ./logs

## 5. Evaluación Final

In [ ]:
results = {
    "Baseline": model_baseline.evaluate(X_test, y_test, verbose=0),
    "Simple": model_simple.evaluate(X_test, y_test, verbose=0),
    "Deep": model_deep.evaluate(X_test, y_test, verbose=0)
}

df_results = pd.DataFrame(results, index=["MSE", "MAE"]).T
print(df_results)